<a href="https://colab.research.google.com/github/ykitaguchi77/Kaggle/blob/main/H-M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**Open transaction file**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Deep_learning/Kaggle_H&M/transactions_train.csv')

In [ ]:
#Pickle形式で一旦保存（Loadが早くなる）
df.to_pickle('/content/drive/MyDrive/Deep_learning/Kaggle_H&M/transactions_train.pickle') # pickle ファイルに変換

In [4]:
train = pd.read_pickle('/content/drive/MyDrive/Deep_learning/Kaggle_H&M/transactions_train.pickle') # pickleファイルの読み込み
train

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


#**重複を削除したカスタマーリストを作成**

In [10]:
customer_id_list = train.iloc[:, 1]
customer_id_list = customer_id_list[~customer_id_list.duplicated()].reset_index(drop=True)
customer_id_list

0          000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
1          00007d2de826758b65a93dd24ce629ed66842531df6699...
2          00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...
3          0008968c0d451dbc5a9968da03196fe20051965edde741...
4          000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...
                                 ...                        
1362276    fe99a0069d6b3c64c2707d0ce53b9311540917471d82df...
1362277    fecc5f77b5f7ee4570efde9ab05ec94d0de2bf80efb4f6...
1362278    fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...
1362279    fee56cc5315dafb35a4490ccc6f711092cae913550c832...
1362280    ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...
Name: customer_id, Length: 1362281, dtype: object

In [8]:
print(len(customer_id_list)) #1362281人のデータ

1362281


In [13]:
a = train[train.iloc[:,1] == customer_id_list[3]]
len(a)

61

#**重複を削除した商品リストを作成**

In [ ]:
article_list = train.iloc[:, 2]
article_list = article_list[~article_list.duplicated()].reset_index(drop=True)
article_list #商品が104546個もある！！

0         663713001
1         541518023
2         505221004
3         685687003
4         685687004
            ...    
104542    913290001
104543    860135008
104544    539060028
104545    810737015
104546    533261032
Name: article_id, Length: 104547, dtype: int64

In [ ]:
len(article_list)

104547

#**類似商品のクラスタリング**
https://qiita.com/kotai2003/items/55c4f525ff216ca16800

In [14]:
!pip install pycaret
import pycaret
from pycaret.datasets import get_data


     |████████████████████████████████| 301 kB 4.7 MB/s 
     |████████████████████████████████| 6.8 MB 37.2 MB/s 
     |████████████████████████████████| 15.6 MB 41.6 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 114 kB 51.8 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 
     |████████████████████████████████| 274 kB 43.7 MB/s 
     |████████████████████████████████| 1.3 MB 30.9 MB/s 
     |████████████████████████████████| 1.7 MB 36.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.0 MB 27.3 MB/s 
     |████████████████████████████████| 167 kB 52.6 MB/s 
     |████████████████████████████████| 261 kB 53.7 MB/s 
     |████████████████████████████████| 675 kB 51.0 MB/s 
     |████████████████████████████████| 10.9 MB 35.6 MB/s 
     |████████████████

In [11]:
import pandas as pd
import pandas_profiling
df_article = pd.read_csv("/content/drive/MyDrive/Deep_learning/Kaggle_H&M/articles.csv")

#パイロット用に少量のデータをピックアップする
df_article = article.iloc[:5, :]
df_article

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,4,Dark,5,Black,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,3,Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,1,Dusty Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,4,Dark,5,Black,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,3,Light,9,White,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [46]:
for j in range(len(df_article.columns)):
    print(df_article.iloc[0,:].str.contains("Lingeries")[j])

nan
nan
False
nan
False
False
nan
False
nan
False
nan
False
nan
False
nan
False
False
False
nan
False
nan
False
nan
False
False


In [ ]:
product_num = len(df_article)

article_class_list = [0] * product_num
class_list = ["Boys", "Girls", "Ba", ""]

def set_feature(class_list, loc, new_class):
    if class_list[loc] == 0:
        class_list[loc] = new_class
    else:
        pass

for i in range(len(df_article)):
    print(i)
    for j in range(len(df_article.columns)):
        if df_article.iloc[i,:].str.contains("Lingeries")[j] is True:
            set_feature(article_class_list, i, "Lingeries")

In [52]:
article_class_list

[0, 0, 0, 'Lingeries', 'Lingeries']

In [17]:
df_article.iloc[0,:].str.contains("Strap")

article_id                        NaN
product_code                      NaN
prod_name                        True
product_type_no                   NaN
product_type_name               False
product_group_name              False
graphical_appearance_no           NaN
graphical_appearance_name       False
colour_group_code                 NaN
colour_group_name               False
perceived_colour_value_id         NaN
perceived_colour_value_name     False
perceived_colour_master_id        NaN
perceived_colour_master_name    False
department_no                     NaN
department_name                 False
index_code                      False
index_name                      False
index_group_no                    NaN
index_group_name                False
section_no                        NaN
section_name                    False
garment_group_no                  NaN
garment_group_name              False
detail_desc                     False
Name: 0, dtype: object

In [ ]:
article = df_article[["product_type_name","product_group_name","graphical_appearance_name", "colour_group_name","perceived_colour_value_name", "perceived_colour_master_name", "department_name", "index_name", "garment_group_name"]]
article.profile_report()

In [ ]:
data = article.sample(frac=0.95, random_state=786).reset_index(drop=True)
data_unseen = article.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (475, 9)
Unseen Data For Predictions: (25, 9)


In [ ]:
from pycaret.clustering import *
data_clust = setup(data, normalize = True, session_id = 123)

In [ ]:
kmeans = create_model('kmeans',num_clusters = 7 )
print(kmeans)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.159,35.8255,2.3639,0,0,0


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=7, n_init=10, n_jobs=-1, precompute_distances='deprecated',
       random_state=123, tol=0.0001, verbose=0)


In [ ]:
kmean_results = assign_model(kmeans)
kmean_results.head()

,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,garment_group_name,Cluster
0,Underwear Tights,Socks & Tights,Solid,Black,Dark,Black,Tights basic,Lingeries/Tights,Socks and Tights,Cluster 1
1,Trousers,Garment Lower body,Denim,Light Blue,Dusty Light,Blue,Woven bottoms,Ladieswear,Trousers,Cluster 6
2,Top,Garment Upper body,Solid,Grey,Dusty Light,Grey,Kids Boy Jersey Basic,Children Sizes 92-140,Jersey Basic,Cluster 5
3,Hair/alice band,Accessories,Solid,Light Orange,Dusty Light,Orange,Hair Accessories,Ladies Accessories,Accessories,Cluster 3
4,Pyjama jumpsuit/playsuit,Nightwear,All over pattern,Light Red,Medium,Red,Baby Nightwear,Baby Sizes 50-98,"Under-, Nightwear",Cluster 4


In [ ]:
pd.set_option('display.max_rows', 500)
kmean_results

In [ ]:
#PCA plot
plot_model(kmeans)

In [ ]:
#Elbow plot (suggested cluster number)
plot_model(kmeans, plot = 'elbow')

In [ ]:
#Distribution plot
plot_model(kmeans, plot = 'distribution', feature = 'class')

In [ ]:
#Predictions
unseen_predictions = predict_model(kmeans, data=data_unseen)
unseen_predictions.head()

In [ ]:
"""
種類 → product type name, product group name：　下着（上下）、アンダー、シャツ、ジャケット、アクセサリー
男性物、女性物、子供、赤ちゃん

どのように分類するか？
Mens/Ladies/Kids Boy/Kids Girl/Babies
Product type name/Product group name/Garment group name
Colour  perceived color master, perceived color name 
Childrenはサイズが分かれている

このあたりで大まかに分類し、購入の傾向をつかむ
→後で細分化する
"""